In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

reset_graph()

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [2]:
# load the training set
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("datasets/")
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

Extracting datasets/train-images-idx3-ubyte.gz
Extracting datasets/train-labels-idx1-ubyte.gz
Extracting datasets/t10k-images-idx3-ubyte.gz
Extracting datasets/t10k-labels-idx1-ubyte.gz


In [3]:
height = 28
width = 28
channels = 1
n_inputs = height * width

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

In [4]:
conv1_fmaps = 6
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv3_fmaps = 16
conv3_ksize = 5
conv3_stride = 1
conv3_pad = "VALID"

conv5_fmaps = 120
conv5_ksize = 5
conv5_stride = 1
conv5_pad = "VALID"

n_fc6 = 84
outputs = 10 

he_init = tf.contrib.layers.variance_scaling_initializer()

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize, 
                         strides=conv1_stride,padding=conv1_pad,activation=tf.tanh, name="conv1")

print(conv1.shape)

pool2 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

print(pool2.shape)


conv3 = tf.layers.conv2d(pool2, filters= conv3_fmaps, kernel_size=conv3_ksize, 
                         strides=conv3_stride,padding=conv3_pad,activation=tf.tanh, name="conv3")

print(conv3.shape)

pool4 = tf.nn.avg_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

print(pool4.shape)

conv5 = tf.layers.conv2d(pool4, filters= conv5_fmaps, kernel_size=conv5_ksize, 
                         strides=conv5_stride,padding=conv5_pad,activation=tf.tanh, name="conv5")

print(conv5.shape)

conv5_flat = tf.reshape(conv5, shape=[-1, conv5_fmaps * 1 * 1]) # can be guessed by pool.shape
    
fc6 = tf.layers.dense(conv5_flat, n_fc6, activation=tf.tanh, kernel_initializer = he_init, name="fc6")

logits = tf.layers.dense(fc6, outputs, kernel_initializer = he_init, name="logits")

Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

(?, 28, 28, 6)
(?, 14, 14, 6)
(?, 10, 10, 16)
(?, 5, 5, 16)
(?, 1, 1, 120)


In [5]:
%%time

n_epochs = 10
batch_size = 100

init = tf.global_variables_initializer()
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
with tf.Session() as session:
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            session.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images[:2000], y: mnist.test.labels[:2000]})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.96 Test accuracy: 0.9445
1 Train accuracy: 0.97 Test accuracy: 0.9605
2 Train accuracy: 0.96 Test accuracy: 0.9725
3 Train accuracy: 0.99 Test accuracy: 0.9735
4 Train accuracy: 0.99 Test accuracy: 0.9775
5 Train accuracy: 0.99 Test accuracy: 0.9805
6 Train accuracy: 1.0 Test accuracy: 0.9795
7 Train accuracy: 0.99 Test accuracy: 0.9755
8 Train accuracy: 0.99 Test accuracy: 0.9785
9 Train accuracy: 1.0 Test accuracy: 0.9785
CPU times: user 36.8 s, sys: 6.43 s, total: 43.2 s
Wall time: 41.5 s
